In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import time

In [2]:
# data preparation
data = pd.read_csv('data/training.csv')
m_data = data.shape[0]
df_train, df_test = data[0:int(0.7*m_data)], data[int(0.7*m_data)+1:m_data]
df_test = df_test.reset_index(drop=True)

df = pd.read_csv('data/race-result-horse.csv')
m, n = np.shape(df)
m_train, _ = np.shape(df_train)
m_test, _ = np.shape(df_test)

#produce dicts for horse,jockey,trainer
horse=list()
jockey=list()
trainer=list()
for i in range(len(df)):
    if df.horse_name[i] not in horse:
        horse.append(df.horse_name[i])
    if df.jockey[i] not in jockey:
        jockey.append(df.jockey[i])
    if df.trainer[i] not in jockey:
        trainer.append(df.trainer[i])
train_horse=np.zeros((m_train,1))
train_jockey=np.zeros((m_train,1))
train_trainer=np.zeros((m_train,1))

In [3]:
data_test = pd.read_csv('data/testing.csv')
data_test

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,...,finish_time,win_odds,running_position_5,running_position_6,race_id,recent_6_runs,recent_ave_rank,jockey_ave_rank,trainer_ave_rank,race_distance
0,3,10.0,PADDINGTON,A107,A Badel,W Y So,125,1144,12,1,...,0.57.79,90.0,NaN,NaN,2016-327,NaN,7.00,7,7,1000
1,4,2.0,CHEER WIN,V347,K K Chiong,P F Yiu,128,1124,4,1,...,0.57.82,3.7,NaN,NaN,2016-327,4/11/2/1/5/8,5.17,7,7,1000
2,5,1.0,BEAUTY CONNECTION,T415,J Moreira,A S Cruz,133,1131,9,2-1/2,...,0.58.07,5.9,NaN,NaN,2016-327,12/12/4/2/9/4,7.17,4,7,1000
3,6,5.0,SECRET AGENT,P388,S de Sousa,K L Man,132,1064,11,3,...,0.58.12,11.0,NaN,NaN,2016-327,2/9/3/4/1/3,3.67,6,7,1000
4,7,11.0,ST YAZIN,N409,O Murphy,T K Ng,121,1075,5,3-1/4,...,0.58.18,19.0,NaN,NaN,2016-327,9/1/6/1/9/5,5.17,8,8,1000
5,8,4.0,OCEAN ROAR,N317,M L Yeung,A Lee,130,1203,1,4,...,0.58.28,17.0,NaN,NaN,2016-327,11/11/8/10/9/12,10.17,7,7,1000
6,9,7.0,WINFULL PATROL,P063,K C Leung,R Gibson,129,1237,2,5-1/4,...,0.58.48,18.0,NaN,NaN,2016-327,10/11/9/10/4/2,7.67,7,7,1000
7,10,12.0,FANTASTIC FABIO,V364,C Y Ho,C H Yip,114,1141,3,9,...,0.59.11,87.0,NaN,NaN,2016-327,11/10/12/7/9,9.80,7,7,1000
8,11,8.0,LYRIC ACE,S047,K C Ng,D Cruz,123,1071,7,15-1/4,...,1.00.08,47.0,NaN,NaN,2016-327,11/5/1/7/10/2,6.00,9,7,1000
9,1,5.0,POWERMAX,A009,N Callan,R Gibson,126,1124,9,-,...,1.10.69,6.0,NaN,NaN,2016-328,4/3/3,3.33,6,7,1200


In [4]:
# prepare training data
for i in range(m_train):
    train_horse[i,0] = horse.index(df_train.horse_name[i])
    train_jockey[i,0] = jockey.index(df_train.jockey[i])
    train_trainer[i,0] = trainer.index(df_train.trainer[i])
actual_weight = df_train.actual_weight.reshape((m_train,1))
declared_weight = df_train.declared_horse_weight.reshape((m_train,1))
draw = df_train.draw.reshape((m_train,1))
win_odds = df_train.win_odds.reshape((m_train,1))
race_distance = df_train.race_distance.reshape((m_train,1))

# we use horse, jockey, trainer, actual weight, declared weight, win odds, race distance as independent variables
X_train = np.hstack((train_horse, train_jockey, train_trainer, actual_weight,
                     declared_weight, draw, win_odds, race_distance)) 
y_train = df_train.finishing_position

# prepare test data
test_horse=np.zeros((m_test,1))
test_jockey=np.zeros((m_test,1))
test_trainer=np.zeros((m_test,1))

#print(df_test.horse_name[])
for i in range(len(df_test)):
    test_horse[i,0] = horse.index(df_test.horse_name[i])
    test_jockey[i,0] = jockey.index(df_test.jockey[i])
    test_trainer[i,0] = trainer.index(df_test.trainer[i])
actual_weight_test = df_test.actual_weight.reshape((m_test,1))
declared_weight_test = df_test.declared_horse_weight.reshape((m_test,1))
draw_test = df_test.draw.reshape((m_test,1))
win_odds_test = df_test.win_odds.reshape((m_test,1))
race_distance_test = df_test.race_distance.reshape((m_test,1))
X_test = np.hstack((test_horse,test_jockey,test_trainer,actual_weight_test,
                    declared_weight_test, draw_test, win_odds_test, race_distance_test))
y_test = df_test.finishing_position

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  if __name__ == '__main__':
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load

In [ ]:
#3.1.1 Logistic
start_time = time.time()
lr_model = LogisticRegression(C=0.1, random_state=0)
lr_model.fit(X_train,y_train)
lr_result = lr_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
# print(lr_result.shape)
lr_score = lr_model.score(X_test,y_test)
print(lr_score)


In [ ]:
print('start')
parameters = {'penalty': ['l1', 'l2'], 'C': [0.06, 0.08, 0.1, 1], 'random_state': [0]}
lr = LogisticRegression()
clf = GridSearchCV(lr, parameters)
clf.fit(X_train, y_train)
print('finish')

In [ ]:
# print(clf.cv_results_)
print(clf.best_estimator_)
print(clf.best_params_)
lr_new = clf.best_estimator_
lr_new.fit(X_train,y_train)
print(lr_new.score(X_test, y_test))

In [ ]:
# 3.1.2 Naïve Bayes
start_time = time.time()
nb_model = sklearn.naive_bayes.GaussianNB()
nb_model.fit(X_train, y_train)
nb_result=nb_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
nb_score = nb_model.score(X_test, y_test)
print(nb_score)


In [5]:
from naive_bayes import NaiveBayes

In [6]:

clf = NaiveBayes()
clf = clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
# 3.1.3 SVM
# first normalize the data


from sklearn import svm
svm_model = svm.SVC(kernel='rbf',random_state=0,gamma=0.005,C=0.7)
#print('model')
#svm_model.fit(X_train, y_train)
#print('fit')
#svm_score = svm_model.score(X_test, y_test)
#print(svm_score)
start_time = time.time()
#print('model')
svm_model.fit(X_train, y_train)
#print('fit')
svm_result=svm_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
svm_score = svm_model.score(X_test, y_test)
#print(svm_model.coef_)
#print(svm_model.predict(X_test))
print(svm_score)


In [ ]:
y_train.shape
X_train_normalized

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_X=StandardScaler()
scaler_X.fit(X_train)
X_train_normalized=scaler_X.transform(X_train)
scaler_X.fit(X_test)
X_test_normalized=scaler_X.transform(X_test)

scaler_y=StandardScaler()
scaler_y.fit(y_train.reshape(-1, 1))
y_train_normalized=scaler_y.transform(y_train.reshape(-1, 1))
scaler_y.fit(y_test.reshape(-1, 1))
y_test_normalized=scaler_y.transform(y_test.reshape(-1, 1))

In [ ]:
parameters = {'kernel':['rbf', 'poly'], 'C':[0.1, 0.5, 1], 'gamma': ['auto', 0.01, 0.1], 
              'random_state': [0], 'max_iter': [100, 500, 1000]}
svc = svm.SVC()
print('start training')
clf = GridSearchCV(svc, parameters)
clf.fit(X_train_normalized, y_train_normalized)
print('finish')
sorted(clf.cv_results_.keys())

In [ ]:
print(clf.best_params_)
svm_new = clf.best_estimator_
svm_new.fit(X_train_normalized,y_train_normalized)
print(svm_new.score(X_test_normalized, y_test_normalized))

In [ ]:
print('start')
parameters = {'n_estimators': [30, 50, 100], 'max_features': ['sqrt', 'log2', None], 
              'max_depth': [None,1, 2, 5], 'random_state': [0]}
raf = RandomForestClassifier()
clf = GridSearchCV(raf, parameters)
clf.fit(X_train, y_train)
print('finish')
sorted(clf.cv_results_.keys())


In [ ]:
print(clf.best_params_)
raf_new = clf.best_estimator_
raf_new.fit(X_train,y_train)
print(raf_new.score(X_test, y_test))
print(clf.best_estimator_)
print(raf_new.feature_importances_)

In [ ]:
#3.1.4 Random Forest

start_time = time.time()
rf_model=RandomForestClassifier(max_depth=2,random_state=0)

rf_model.fit(X_train,y_train)
rf_result=rf_model.predict(X_test)
print('training time: ', time.time()-start_time, 'secends')
#print(rf_model.predict(X_test))
print(rf_model.score(X_test,y_test))

In [ ]:
#3.2
from collections import Counter
#The lr matrix
horse_win_lr=np.zeros((len(lr_result),1))
horse_top3_lr=np.zeros((len(lr_result),1))
horse_top50percent_lr=np.zeros((len(lr_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(lr_result)):
    if lr_result[i] == 1:
         horse_win_lr[i]=1
    if lr_result[i] <= 3:
         horse_top3_lr[i] = 1
    if lr_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
         horse_top50percent_lr[i] = 1
headers=['RaceID','HorseID','HorseWin','HorseRankTop3','HorseRankTop50Percent']
import csv
with open('lr_predictions.csv','w') as f1:
     lr_csv=csv.writer(f1)
     lr_csv.writerow(headers)
     for i in range(len(lr_result)):
         lr_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_lr[i][0],horse_top3_lr[i][0],horse_top50percent_lr[i][0]])
#the naive bayesian matrix
horse_win_nb=np.zeros((len(nb_result),1))
horse_top3_nb=np.zeros((len(nb_result),1))
horse_top50percent_nb=np.zeros((len(nb_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(nb_result)):
    if nb_result[i] == 1:
        horse_win_nb[i]=1
    if nb_result[i] <= 3:
        horse_top3_nb[i] = 1
    if nb_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
        horse_top50percent_nb[i] = 1
with open('nb_predictions.csv','w') as f2:
     nb_csv=csv.writer(f2)
     nb_csv.writerow(headers)
     for i in range(len(nb_result)):
         nb_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_nb[i][0],horse_top3_nb[i][0],horse_top50percent_nb[i][0]])
#the svm matrix
horse_win_svm=np.zeros((len(svm_result),1))
horse_top3_svm=np.zeros((len(svm_result),1))
horse_top50percent_svm=np.zeros((len(svm_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(svm_result)):
    if svm_result[i] == 1:
        horse_win_svm[i]=1
    if svm_result[i] <= 3:
        horse_top3_svm[i] = 1
    if svm_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
        horse_top50percent_svm[i] = 1
with open('svm_predictions.csv','w') as f3:
    svm_csv=csv.writer(f3)
    svm_csv.writerow(headers)
    for i in range(len(svm_result)):
        svm_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_svm[i][0],horse_top3_svm[i][0],horse_top50percent_svm[i][0]])
#the rf matrix
horse_win_rf=np.zeros((len(rf_result),1))
horse_top3_rf=np.zeros((len(rf_result),1))
horse_top50percent_rf=np.zeros((len(rf_result),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(rf_result)):
    if rf_result[i] == 1:
        horse_win_rf[i]=1
    if rf_result[i] <= 3:
        horse_top3_rf[i] = 1
    if rf_result[i]<= np.floor(count_of_race_participation[df_test.race_id[i]]/2):
        horse_top50percent_rf[i] = 1
with open('rf_predictions.csv','w') as f4:
    rf_csv=csv.writer(f4)
    rf_csv.writerow(headers)
    for i in range(len(rf_result)):
        rf_csv.writerow([df_test.race_id[i],df_test.horse_id[i],horse_win_rf[i][0],horse_top3_rf[i][0],horse_top50percent_rf[i][0]])

In [ ]:
#3.3
#get actual results from y_test
horse_win_actual=np.zeros((len(y_test),1))
horse_top3_actual=np.zeros((len(y_test),1))
horse_top50percent_actual=np.zeros((len(y_test),1))
count_of_race_participation=Counter(df_test.race_id)
for i in range(len(y_test)):
    if y_test[i]==1:
         horse_win_actual[i]=1
    if y_test[i]<=3:
         horse_top3_actual[i]=1
    if y_test[i]<=np.floor(count_of_race_participation[df_test.race_id[i]]/2):
         horse_top50percent_actual[i]=1
#lr model
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr horse_win
    if (horse_win_lr[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_lr[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_lr[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_lr[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
print("Recall of logistic model horse_win prediction= ", TP/(TP+FN))
print("Precision of logistic model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_lr[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_lr[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_lr[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_lr[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
print("Recall of logistic model horse_top3 prediction= ", TP/(TP+FN))
print("Precision of logistic model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_lr[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_lr[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_lr[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_lr[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
print("Recall of logistic model horse_top50percent prediction= ", TP/(TP+FN))
print("Precision of logistic model horse_top50percent prediction= ", TP/(TP+FP))
#naive bayesian
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#nb horse_win
    if (horse_win_nb[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_nb[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_nb[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_nb[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
print("Recall of naive bayesian model horse_win prediction= ", TP/(TP+FN))
print("Precision of naive bayesian model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_nb[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_nb[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_nb[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_nb[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
print("Recall of naive bayesian model horse_top3 prediction= ", TP/(TP+FN))
print("Precision of naive bayesian model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_nb[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_nb[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_nb[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_nb[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
print("Recall of naive bayesian model horse_top50percent prediction= ", TP/(TP+FN))
print("Precision of naive bayesian model horse_top50percent prediction= ", TP/(TP+FP))
#svm
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#nb horse_win
    if (horse_win_svm[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_svm[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_svm[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_svm[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
if ((TP+FN)==0 or (TP+FP)==0):
    print("Recall of svm model horse_win prediction and Precision of svm model horse_win prediction= 0")
else:
    print("Recall of svm model horse_win prediction= ", TP/(TP+FN))
    print("Precision of svm model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_svm[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_svm[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_svm[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_svm[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
if ((TP+FN)==0 or (TP+FP)==0):
     print("Recall of svm model horse_top3 prediction and Precision of svm model horse_top3 prediction= 0")
else:
     print("Recall of svm model horse_top3 prediction= ", TP/(TP+FN))
     print("Precision of svm model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_svm[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_svm[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_svm[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_svm[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
if ((TP+FN)==0 or (TP+FP)==0):
     print("Recall of svm model horse_top50percent prediction and Precision of svm model horse_top50percent prediction= 0")
else:
     print("Recall of svm model horse_top50percent prediction= ", TP/(TP+FN))
     print("Precision of svm model horse_top50percent prediction= ", TP/(TP+FP))
#rf model
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#nb horse_win
    if (horse_win_rf[i]==1 and horse_win_actual[i]==1):
        TP=TP+1
    if (horse_win_rf[i]==1 and horse_win_actual[i]==0):
        FP=FP+1
    if (horse_win_rf[i]==0 and horse_win_actual[i]==1):
        FN=FN+1
    if (horse_win_rf[i]==0 and horse_win_actual[i]==0):
        TN=TN+1
print("Recall of rf model horse_win prediction= ", TP/(TP+FN))
print("Precision of rf model horse_win prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top3
    if (horse_top3_rf[i]==1 and horse_top3_actual[i]==1):
        TP=TP+1
    if (horse_top3_rf[i]==1 and horse_top3_actual[i]==0):
        FP=FP+1
    if (horse_top3_rf[i]==0 and horse_top3_actual[i]==1):
        FN=FN+1
    if (horse_top3_rf[i]==0 and horse_top3_actual[i]==0):
        TN=TN+1
print("Recall of rf model horse_top3 prediction= ", TP/(TP+FN))
print("Precision of rf model horse_top3 prediction= ", TP/(TP+FP))
TP=0;FP=0;FN=0;TN=0
for i in range(len(X_test)):#lr_top50percent
    if (horse_top50percent_rf[i]==1 and horse_top50percent_actual[i]==1):
        TP=TP+1
    if (horse_top50percent_rf[i]==1 and horse_top50percent_actual[i]==0):
        FP=FP+1
    if (horse_top50percent_rf[i]==0 and horse_top50percent_actual[i]==1):
        FN=FN+1
    if (horse_top50percent_rf[i]==0 and horse_top50percent_actual[i]==0):
        TN=TN+1
print("Recall of rf model horse_top50percent prediction= ", TP/(TP+FN))
print("Precision of rf model horse_top50percent prediction= ", TP/(TP+FP))
